# Imports

Before starting the import into Spotify you need to go to Spotify Developers and make a new App. From there onwards you need to follow this Notebook.

In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
# Make a file in your directory containing username, client_id and client_secret
import keys

In [4]:
songs = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists.csv')

In [9]:
scope='playlist-modify-public'
username= keys.username
redirectUri = 'http://localhost:7000/callback'
client_id=keys.client_id
client_secret=keys.client_secret
playlist_name = 'My Second Playlist'
playlist_description = 'App for my bootcamp project Moosaic. Create playlists through clustering.'

In [10]:
token = SpotifyOAuth(scope=scope,username=username,client_id=client_id,
                     client_secret=client_secret, redirect_uri=redirectUri)

In [11]:
spotifyObject = spotipy.Spotify(auth_manager=token)

In [12]:
try:
    playlist_id = spotifyObject.user_playlist_create(user=username, name=playlist_name, 
                                                     description=playlist_description, public=True)
    print(f"Playlist created with ID: {playlist_id}")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: [Errno 48] Address already in use


In [ ]:
playlist_name='our_little_playlist'
playlist_description='10 very diferent songs'

In [ ]:
playlist_id

In [ ]:
list_of_songs=[]
for i, row in songs.iterrows():
    list_of_songs.append(f'spotify:track:{row["id"]}')

In [ ]:
list_of_songs

In [ ]:
spotifyObject.user_playlist_add_tracks(user=username,playlist_id=playlist_id['id'],tracks=list_of_songs)

# Try otherwise

In [14]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
import keys

songs = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Moosic/Data/playlists.csv')

# Spotify API credentials (You'll get these from the Spotify Developer Dashboard)
SPOTIPY_CLIENT_ID = keys.client_id
SPOTIPY_CLIENT_SECRET = keys.client_secret
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback'

# Setting up spotipy with user authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    scope="playlist-modify-public"  # This scope allows creating public playlists
))

# Your user ID (You can also retrieve this programmatically after authentication)
username = keys.username

# Details for the new playlist
playlist_name = "My New Playlist"
playlist_description = "This is a playlist created using the Spotify Web API."

try:
    # Create a new playlist
    playlist = sp.user_playlist_create(user=username, name=playlist_name, 
                                       description=playlist_description, public=True)
    print(f"Playlist created with ID: {playlist['id']}")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: [Errno 48] Address already in use
